## Preparacion de notebook y datos:

In [44]:
pip install eli5

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')


import pandas as pd
import numpy as np


from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

In [33]:
import keras
from keras.models import Sequential
from tensorflow.keras import metrics
from tensorflow.keras.metrics import AUC, TruePositives, FalsePositives, FalseNegatives
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization, PReLU, ELU, LeakyReLU
from keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras import optimizers, initializers
from tensorflow.keras.optimizers import SGD, Adam
from keras.initializers import glorot_uniform, he_uniform, uniform, Constant
from keras.constraints import maxnorm
from keras.regularizers import l2

from keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint

In [34]:
desc_path = 'Descriptores_nr_er.csv' #Este archivo se obtuvo con la notebook tox21_DescriptorsCalc
descriptores = pd.read_csv(desc_path)
descriptores

,Unnamed: 0,abonds,atoms,bonds,cansmi,cansmiNS,dbonds,formula,HBA1,HBA2,...,MW,nF,rotors,s,sbonds,smarts,tbonds,title,TPSA,CHARGE
0,0,11.0,42.0,44.0,0.0,0.0,0.0,0.0,19.0,3.0,...,378.312159,6.0,4.0,0.0,33.0,0.0,0.0,0.0,45.15,0
1,1,18.0,69.0,73.0,0.0,0.0,0.0,0.0,39.0,4.0,...,457.603800,0.0,7.0,0.0,55.0,0.0,0.0,0.0,30.93,0
2,2,17.0,30.0,32.0,0.0,0.0,0.0,0.0,18.0,7.0,...,253.262640,0.0,1.0,0.0,15.0,0.0,0.0,0.0,129.62,0
3,3,12.0,59.0,61.0,0.0,0.0,4.0,0.0,28.0,4.0,...,494.475279,6.0,7.0,0.0,45.0,0.0,0.0,0.0,48.78,0
4,4,0.0,92.0,91.0,0.0,0.0,0.0,0.0,64.0,3.0,...,474.855700,0.0,25.0,0.0,91.0,0.0,0.0,0.0,27.69,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7692,7692,6.0,21.0,21.0,0.0,0.0,2.0,0.0,12.0,2.0,...,170.232100,0.0,2.0,0.0,13.0,0.0,0.0,0.0,80.74,0
7693,7693,0.0,12.0,12.0,0.0,0.0,1.0,0.0,9.0,3.0,...,102.158140,0.0,0.0,0.0,11.0,0.0,0.0,0.0,56.15,0
7694,7694,0.0,12.0,12.0,0.0,0.0,1.0,0.0,9.0,3.0,...,102.158140,0.0,0.0,0.0,11.0,0.0,0.0,0.0,56.15,0
7695,7695,6.0,32.0,32.0,0.0,0.0,2.0,0.0,19.0,4.0,...,291.260621,0.0,7.0,0.0,24.0,0.0,0.0,0.0,115.41,0


In [35]:
#Read target.
actives_f = open("nr_er_activity.txt", "r") #Este archivo se obtuvo de la columna de Actividad del SDF.
activity = []
for x in actives_f:
  activity.append(x[0])
#activity = activity[1:]
activity = [int(i) for i in activity]
y = np.array(activity)
print(y.shape)
actives_f.close()

(7697,)


In [36]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(descriptores.values, y, test_size =0.1,random_state = 42, stratify=activity)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(6927, 27)
(770, 27)
(6927,)
(770,)


In [37]:
from sklearn.preprocessing import StandardScaler

desc_scaler = StandardScaler()
X_train = desc_scaler.fit_transform(X_train)
Scaled_descriptors = pd.DataFrame(X_train, columns=descriptores.columns)
Scaled_descriptors.head()

,Unnamed: 0,abonds,atoms,bonds,cansmi,cansmiNS,dbonds,formula,HBA1,HBA2,...,MW,nF,rotors,s,sbonds,smarts,tbonds,title,TPSA,CHARGE
0,-0.867422,0.061204,0.681038,0.628320,0.0,0.0,-0.267463,0.0,0.776294,-0.191243,...,0.159438,-0.148362,0.746655,0.0,0.686063,0.0,-0.168185,0.0,-0.474732,0.113714
1,-0.218618,0.061204,-0.696640,-0.695395,0.0,0.0,-0.890852,0.0,-0.664650,-0.746224,...,-0.858691,-0.148362,-0.522930,0.0,-0.697721,0.0,-0.168185,0.0,-0.677815,0.113714
2,0.849975,0.061204,-0.554122,-0.558459,0.0,0.0,-0.267463,0.0,-0.520556,-0.191243,...,-0.568211,-0.148362,-0.311333,0.0,-0.598879,0.0,-0.168185,0.0,-0.421236,0.113714
3,-0.431886,0.061204,0.015952,-0.010714,0.0,0.0,-0.890852,0.0,0.127869,-0.191243,...,-0.283662,-0.148362,0.323460,0.0,0.043592,0.0,-0.168185,0.0,-0.136819,0.113714
4,-1.397894,-0.992992,0.633532,0.719611,0.0,0.0,0.979316,0.0,0.560152,-0.468734,...,0.225386,-0.148362,-0.946125,0.0,0.982589,0.0,-0.168185,0.0,-0.444546,0.113714


In [38]:
#Validation Scale:
X_val = desc_scaler.transform(X_val)

## Preparacion de Modelo final con mejores hiperparámetros:

In [99]:
optimizer = optimizers.Adam(
                learning_rate=0.1,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-7,
                amsgrad=False)

In [164]:
es = EarlyStopping(monitor='val_loss', verbose=1, patience=85, restore_best_weights = True)
reduceLR=ReduceLROnPlateau(monitor='val_loss', 
                            factor=0.2, 
                            patience=10, 
                            verbose=1, 
                            min_delta=0, 
                            cooldown=0, min_lr=0)

In [92]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
y_categorical = enc.fit(y_train.reshape(-1, 1))

Definimos el modelo final utilizado y lo levantamos con la funcion KerasClassifier de sklearn:

In [162]:
def model_base():
    model = Sequential()

    model.add(Dense(512, input_dim = X_train.shape[1], kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512,kernel_regularizer = None, kernel_initializer='he_uniform', activation ='relu'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
    model.add(Dense(512,kernel_regularizer = None,kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
    model.add(Dense(512,kernel_regularizer = None,kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
    model.add(Dense(512,kernel_regularizer = None,kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer='he_uniform', activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optimizer ,metrics=['accuracy'])    
    
    return model


In [163]:
optimizer = optimizers.Adam(
                learning_rate=0.1,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-7,
                amsgrad=False)

In [165]:
my_model = KerasClassifier(build_fn=model_base, batch_size=256, validation_data=(X_val, y_val), epochs=400, verbose=2, shuffle=True, callbacks= [es, reduceLR])    
# my_model.fit(X_train,y_train)

Hacemos un fit del modelo obtenido conla funcion KerasClassifier de Sklearn.

In [166]:
my_model.fit(X_train,y_train)

Train on 6927 samples, validate on 770 samples
Epoch 1/400
 - 2s - loss: 1.1483 - accuracy: 0.7775 - val_loss: 19.5503 - val_accuracy: 0.6013
Epoch 2/400
 - 1s - loss: 0.3926 - accuracy: 0.8691 - val_loss: 3.1866 - val_accuracy: 0.5649
Epoch 3/400
 - 1s - loss: 0.7129 - accuracy: 0.8247 - val_loss: 3.2957 - val_accuracy: 0.1558
Epoch 4/400
 - 1s - loss: 0.3560 - accuracy: 0.8769 - val_loss: 0.9258 - val_accuracy: 0.5701
Epoch 5/400
 - 1s - loss: 0.3107 - accuracy: 0.8841 - val_loss: 0.6911 - val_accuracy: 0.6610
Epoch 6/400
 - 1s - loss: 0.3035 - accuracy: 0.8886 - val_loss: 0.3743 - val_accuracy: 0.8571
Epoch 7/400
 - 1s - loss: 0.3064 - accuracy: 0.8857 - val_loss: 0.3411 - val_accuracy: 0.8753
Epoch 8/400
 - 1s - loss: 0.3048 - accuracy: 0.8888 - val_loss: 0.3158 - val_accuracy: 0.8909
Epoch 9/400
 - 1s - loss: 0.3004 - accuracy: 0.8887 - val_loss: 0.3151 - val_accuracy: 0.8909
Epoch 10/400
 - 1s - loss: 0.3033 - accuracy: 0.8868 - val_loss: 0.3124 - val_accuracy: 0.8948
Epoch 11/40

## Búsqueda de Features de mayor importancia para el modelo:

La función PermutationImportance calcula la media del decrecimiento del score cuando un feature se vuelve ruidoso. La media se realiza sobre el total del numero de iteraciones (n_iter en la función).

In [169]:
# my_model = KerasClassifier(build_fn=basemodel, **sk_params)    
# my_model.fit(X,y)


perm = PermutationImportance(my_model, cv = 'prefit', refit=False, random_state=1, n_iter = 30).fit(X_val,y_val) 

In [114]:
perm.feature_importances_

array([-6.68513156e-02, -8.82695455e-02, -1.75437492e-02, -1.57497167e-02,
        0.00000000e+00,  0.00000000e+00,  3.11595115e-04,  0.00000000e+00,
       -6.65050988e-03,  1.57371270e-05, -4.07874858e-02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.30580385e-02, -9.65000629e-03,
       -2.03008939e-02, -1.10663477e-02, -7.52234672e-04, -1.60927861e-02,
        0.00000000e+00, -9.76646103e-03,  0.00000000e+00,  8.24625456e-04,
        0.00000000e+00, -7.10373914e-03, -4.94145789e-04])

In [170]:
eli5.show_weights(perm, feature_names = Scaled_descriptors.columns.tolist())

Weight,Feature
0.0177 ± 0.0099,abonds
0.0107 ± 0.0117,MP
0.0074 ± 0.0096,HBD
0.0072 ± 0.0093,logP
0.0070 ± 0.0078,rotors
0.0067 ± 0.0076,Unnamed: 0
0.0049 ± 0.0080,MW
0.0033 ± 0.0043,bonds
0.0032 ± 0.0042,atoms
0.0025 ± 0.0069,sbonds


El ranking de importancia obtenido tiene sentido con respecto a la significancia de los features. Los abonds por ejemplo es el número de enlaces aromáticos presentes en el compuesto y es una medida importante para la toxicidad. logP tiene que ver con la lipofilicidad de la molécula y determinante de sus propiedades farmacocinéticas. Por otro lado, descriptores que prácticamente no varían a lo largo de los compuestos como "smarts" o "cansmi" quedaron por debajo del ranking.